In [1]:
from nn import IntrinsicMixin, IntrinsicObserver, IntrinsicTrainingObserver

# PyTorch model and training necessities
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Image datasets and image manipulation
import torchvision
import torchvision.transforms as transforms

# Image display
import matplotlib.pyplot as plt
import numpy as np

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [2]:
# Gather datasets and prepare them for consumption
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Store separate training and validations splits in ./data
training_set = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
validation_set = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

training_loader = torch.utils.data.DataLoader(training_set,
                                              batch_size=4,
                                              shuffle=True,
                                              num_workers=2)


validation_loader = torch.utils.data.DataLoader(validation_set,
                                                batch_size=4,
                                                shuffle=False,
                                                num_workers=2)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

# Extract a batch of 4 images
dataiter = iter(training_loader)
images, labels = next(dataiter)

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

In [3]:
# Default log_dir argument is "runs" - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
writer = SummaryWriter('runs/fashion_mnist_experiment_23')

# Write image data to TensorBoard log dir
writer.add_image('Four Fashion-MNIST Images', img_grid)
writer.flush()

# To view, start TensorBoard on the command line with:
#   tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

In [4]:
class Net(nn.Module, TopologyMixin):
    def __init__(self):
        nn.Module.__init__(self)
        TopologyMixin.__init__(self, writer=writer)
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        self.Filtration(x, label='Input', logging=False)
        self.Dimension(x, label='Dimension Analysis', distances=False, batches=True)
        self.DeltaHyperbolicity(x, label='Hyperbolicity Analysis', distances=False, batches=True)
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        self.Filtration(x, label='Hidden', distances=False, batches=True)
        self.Dimension(x, label='Dimension Analysis', distances=False, batches=True)
        self.DeltaHyperbolicity(x, label='Hyperbolicity Analysis', distances=False, batches=True)
        
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
observer = TopologyTrainingObserver(net, writer=writer, reset=True, log_every_train=1000, log_every_val=1000)

In [7]:
net.Filtration.forward

In [ ]:
net.train(True)

for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    print(observer.step)

    for i, data in enumerate(training_loader, 0):
        # basic training loop
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if i % 1000 == 0:    # Every 1000 mini-batches...
            print('Batch {}'.format(i + 1))
            # Check against the validation set
            running_vloss = 0.0

            # In evaluation mode some model specific operations can be omitted eg. dropout layer
            net.train(False) # Switching to evaluation mode, eg. turning off regularisation
            
            for j, vdata in enumerate(validation_loader, 0):
                vinputs, vlabels = vdata
                voutputs = net(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item()
                
            net.train(True) # Switching back to training mode, eg. turning on regularisation

            avg_loss = running_loss / 10
            avg_vloss = running_vloss / 20

            # Log the running loss averaged per batch
            writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_loss, 'Validation' : avg_vloss },
                            epoch * len(training_loader) + i)

            running_loss = 0.0

print('Finished Training')

In [6]:
observer.topology_modules

In [7]:
observer.topology_modules_forward

In [4]:
class Net(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
from nn.topology import Persistence, DeltaHyperbolicity, Dimension

In [6]:
filtration = Persistence()
dim = Dimension()
dh = DeltaHyperbolicity()

observer = IntrinsicTrainingObserver(
    net, writer=writer, pre_topology=[
        (net.conv1, [
            (filtration, {'label': 'Input'}),
            (dim, {'label': 'Dimension Analysis'}),
            (dh, {'label': 'Hyperbolicity Analysis'})
        ]),
        (net.fc1, [
            (filtration, {'label': 'Hidden'}, lambda x: x.reshape(-1, 16, 4, 4)),
            (dim, {'label': 'Dimension Analysis'}, lambda x: x.reshape(-1, 16, 4, 4)),
            (dh, {'label': 'Hyperbolicity Analysis'}, lambda x: x.reshape(-1, 16, 4, 4))
        ])
    ],
    log_every_val=1000, log_every_train=1000
)

In [6]:
Filtration = Persistence()
Dimension = IntrinsicDimension()
DH = DeltaHyperbolicity()

observer = TopologyObserver(
    net, writer=writer, pre_topology=[
        (net.conv1, [
            (Filtration, {'label': 'Input'}),
            (Dimension, {'label': 'Dimension Analysis'}),
            (DH, {'label': 'Hyperbolicity Analysis'})
        ]),
        (net.fc1, [
            (Filtration, {'label': 'Hidden'}, lambda x: x.reshape(-1, 16, 4, 4)),
            (Dimension, {'label': 'Dimension Analysis'}, lambda x: x.reshape(-1, 16, 4, 4)),
            (DH, {'label': 'Hyperbolicity Analysis'}, lambda x: x.reshape(-1, 16, 4, 4))
        ])
    ]
)

In [6]:
### Example

### Example

In [7]:
# Log Validation Results
net.eval()
vdata = next(iter(validation_loader))
vinputs, vlabels = vdata

dimension = IntrinsicDimension()
dh = DeltaHyperbolicity()
# voutputs = net(vinputs)
# vloss = criterion(voutputs, vlabels)

In [12]:
dh(vinputs)

In [11]:
dimension(vinputs)

In [13]:
from utils.math import compute_unique_distances

batch = vinputs[0]
from nn.functional.dimension import mle, mm

d = compute_unique_distances(batch).detach().numpy()
mle(d)

In [15]:
from scipy.spatial import distance_matrix
def pairwise_dist(bc: np.array):
    return [
        distance_matrix(bc[:, dim], bc[:, dim]) for dim in range(len(bc[0]))
    ]

bc = np.array([dgrm.betti[0] for dgrm in Filtration.diagrams])
pairwise_dist(bc)

In [13]:
from nn.functional.homology import pairwise_dist
pairwise_dist(np.array(bc))

In [8]:
Filtration.diagrams

In [6]:
Filtration = Persistence()
Dimension = IntrinsicDimension()
DH = DeltaHyperbolicity()

observer = TopologyTrainingObserver(
    net, writer=writer, pre_topology=[
        (net.conv1, [
            (Filtration, {'label': 'Input'}),
            (Dimension, {'label': 'Dimension Analysis'}),
            (DH, {'label': 'Hyperbolicity Analysis'})
        ]),
        (net.fc1, [
            (Filtration, {'label': 'Hidden'}, lambda x: x.reshape(-1, 16, 4, 4)),
            (Dimension, {'label': 'Dimension Analysis'}, lambda x: x.reshape(-1, 16, 4, 4)),
            (DH, {'label': 'Hyperbolicity Analysis'}, lambda x: x.reshape(-1, 16, 4, 4))
        ])
    ],
    log_every_train=1000,
    log_every_val=1000
)

In [7]:
net.conv1._forward_pre_hooks

In [7]:
net.train(True)

for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0

    for i, data in enumerate(training_loader, 0):
        # basic training loop
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if i % 1000 == 0:    # Every 1000 mini-batches...
            print('Batch {}'.format(i + 1))
            # Check against the validation set
            running_vloss = 0.0

            # In evaluation mode some model specific operations can be omitted eg. dropout layer
            net.train(False) # Switching to evaluation mode, eg. turning off regularisation
            
            for j, vdata in enumerate(validation_loader, 0):
                vinputs, vlabels = vdata
                voutputs = net(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item()
                
            net.train(True) # Switching back to training mode, eg. turning on regularisation

            avg_loss = running_loss / 10
            avg_vloss = running_vloss / 20

            # Log the running loss averaged per batch
            writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_loss, 'Validation' : avg_vloss },
                            epoch * len(training_loader) + i)

            running_loss = 0.0

print('Finished Training')

In [10]:
ret1, ret2 = filtration.divergence(filtration.diagrams)

In [22]:
observer.train_epoch_information[-1][(5884511808, 'Input')][0].sample.shape

In [8]:
inputs, labels = next(iter(training_loader))
inputs.shape

In [12]:
x = inputs.transpose(1, 2).transpose(2, 3).detach().numpy()

In [14]:
from utils.math import unique_points
unique_points(x[0]).shape

In [10]:
observer.val_epoch_information

In [7]:
Filtration._forward_hooks